# ВКР на политфаке: how to?

## Part I: they see me parsin'
Эта часть посвящена сбору и первичной обработке данных для исследования

*Никита Смирнов, Анна Иванешкина, БПТ-181*

**0. Подгрузка всех необходимых пакетов**

In [1]:
import requests
import numpy as np
import pandas as pd 
import time

from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs

**1. Сбор данных: requests**

In [2]:
# Создадим функцию, которая собирает изначальные данные со страницы конкретной ОП

def parse(program, page):


    url = 'https://www.hse.ru/ba/' + str(program) + '/students/diplomas/' + 'page' + str(page) + '.html'
    response = requests.get(url)
    tree = bs(response.content, 'html.parser')

    works = tree.find('div', {'class' : 'education-content education-content--no_indent'})
    works = works.find_all('div', {'class' : 'edu-programm__item small'})

    data = []

    for item in works:
        try:
            supervisor = item.find('div', {'data-title': 'Руководитель'}).a.text
            superlink = item.find('div', {'data-title': 'Руководитель'}).a.get('href').strip('//')
        except AttributeError:
            supervisor = item.find('div', {'data-title': 'Руководитель'}).text.strip()
            superlink = ''
    
        year = item.find('div', {'data-title': 'Год защиты'}).text
        student = item.find('div', {'data-title': 'ФИО студента'}).text
        
        try:
            text = item.find('div', {'data-title': 'Текст работы'}).span.get('class')
            text = 1
        except AttributeError:
            text = 0

        papername = item.find('div', {'data-title': 'Название работы'}).text
        annotlink =  'https://' + item.find('div', {'data-title': 'Название работы'}).a.get('href').strip('//')

        vkr = [papername, annotlink, text, student, year, supervisor, superlink]
        data.append(vkr)

    data = np.array(data)
    data = pd.DataFrame(data, columns = ['papername', 'annotlink', 'text', 'student', 'year', 'supervisor', 'superlink'])
    return data


In [3]:
# Пройдемся этой функцией по всем ОП ФСН и объединим результаты в один датафрейм: 

polit = pd.concat([parse('political', i) for i in range(1, 16)])
polit['program'] = 'Политология'
soc = pd.concat([parse('soc', i) for i in range(1, 24)])
soc['program'] = 'Социология'
gmu = pd.concat([parse('gmu', i) for i in range(1, 22)])
gmu['program'] = 'ГМУ'
psy = pd.concat([parse('psy', i) for i in range(1, 15)])
psy['program'] = 'Психология'

programs = [polit, soc, gmu, psy]
fsn = pd.concat(programs)
fsn.head()

,papername,annotlink,text,student,year,supervisor,superlink,program
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология


In [2]:
# Теперь пройдемся по всем строкам этой таблицы, и соберем аннотации и оценки для тех ВКР, где они существуют. 
# Определим для этого специальную функцию. 
# Обратим внимание на то, что СЕЛЕНИУМ [будь он проклят] может подвиснуть, застрахуемся от этого. 

def getinfo(link):

    try:
        time.sleep(0.3)
        br.get(link)
        time.sleep(0.3)

        mark = bs(br.page_source).find_all('span', {'class':'vkr-card__value'})[2].get_text()
        annot = bs(br.page_source).find('div', {'class':'vkr-content__text'}).get_text()
    
    except:
        mark = np.NaN
        annot = np.NaN

    return [mark, annot]

In [5]:
# Пройдемся этой функцией по датафрейму:

br = wd.Chrome(r'C:\Users\dramm\Desktop\chromedriver.exe')

fsn['markannot'] = fsn['annotlink'].apply(getinfo)
fsn['mark'] = fsn['markannot'].str[0]
fsn['annot'] = fsn['markannot'].str[1]

fsn.head()

,papername,annotlink,text,student,year,supervisor,superlink,program,markannot,mark,annot
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология,"[8, Данная работа посвящена изучению того, как...",8,"Данная работа посвящена изучению того, какие ф..."
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,"[2020, Работа посвящена изучению террористичес...",2020,Работа посвящена изучению террористической акт...
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,"[2020, Исследование посвящено изучению взаимос...",2020,Исследование посвящено изучению взаимосвязей м...
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология,"[8, Данное исследование фокусируется на выявле...",8,Данное исследование фокусируется на выявлении ...
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология,"[2020, В настоящей работе целью исследования я...",2020,В настоящей работе целью исследования является...


In [6]:
# Пройдемся по misparsed-строкам: 
empty = fsn[fsn['mark'].isnull()]

br = wd.Chrome(r'C:\Users\dramm\Desktop\chromedriver.exe')

empty['markannot'] = empty['annotlink'].apply(getinfo)
empty['mark'] = empty['markannot'].str[0]
empty['annot'] = empty['markannot'].str[1]

In [13]:
# Теперь составим итоговый датафрейм: 

vkr_list = [fsn, empty]
vkr = pd.concat(vkr_list)
vkr.dropna(inplace=True)

vkr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2101 entries, 0 to 20
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   papername   2101 non-null   object
 1   annotlink   2101 non-null   object
 2   text        2101 non-null   object
 3   student     2101 non-null   object
 4   year        2101 non-null   object
 5   supervisor  2101 non-null   object
 6   superlink   2101 non-null   object
 7   program     2101 non-null   object
 8   markannot   2101 non-null   object
 9   mark        2101 non-null   object
 10  annot       2101 non-null   object
dtypes: object(11)
memory usage: 197.0+ KB


In [14]:
# Исправим типы данных и некорректно спарсившиеся аннотации и оценки: 

def markcheck(mark):
    try: 
        mark = int(mark)
        if mark > 10:
            mark = np.NaN
    except: 
        mark = np.NaN
    return mark 

vkr['mark'] = vkr['mark'].apply(markcheck)

def annotcheck(annot): 
    if annot == '-' or annot == 'а' or annot == '_' or annot == '':
        annot = np.NaN
    return annot

vkr['annot'] = vkr['annot'].apply(annotcheck)

vkr[['text', 'year']] = vkr[['text', 'year']].astype('int')

vkr.drop(columns=['markannot'], inplace=True)

vkr.head()

,papername,annotlink,text,student,year,supervisor,superlink,program,mark,annot
0,"""Абсурдные"" электоральные акторы: происхождени...",https://www.hse.ru/edu/vkr/364642404,1,Коротеева Полина Петровна,2020,Каспэ Святослав Игоревич,www.hse.ru/org/persons/67280,Политология,8.0,"Данная работа посвящена изучению того, какие ф..."
1,Взаимосвязь между стабильностью политического ...,https://www.hse.ru/edu/vkr/366243110,0,Минина Анастасия Алексеевна,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Работа посвящена изучению террористической акт...
2,Взаимосвязь между электронным правительством и...,https://www.hse.ru/edu/vkr/366241655,0,Науширванов Тимур Олегович,2020,Розенберг Дина Яновна,www.hse.ru/org/persons/64235685,Политология,NaN,Исследование посвящено изучению взаимосвязей м...
3,Взаимосвязь социально-политической напряженнос...,https://www.hse.ru/edu/vkr/366244578,1,Локаева Екатерина Андреевна,2020,Туровский Ростислав Феликсович,www.hse.ru/org/persons/67524,Политология,8.0,Данное исследование фокусируется на выявлении ...
4,Взаимосвязь уровня коррупции и гендерного нера...,https://www.hse.ru/edu/vkr/366242960,0,Макарова Анастасия Вадимовна,2020,Сальникова Дарья Вячеславовна,www.hse.ru/staff/salnikova,Политология,NaN,В настоящей работе целью исследования является...


Восторг! Сохраним наши данные в CSV, чтобы дальше с ними работать :)

In [16]:
vkr.to_csv(r'C:\Users\dramm\Desktop\vkr.csv')